In [8]:
from lively import  Solver,MoveShape,AddShape,Translation, Rotation,Transform,SmoothnessMacroObjective, CollisionAvoidanceObjective, JointLimitsObjective, BoxShape, CollisionSettingInfo, ScalarRange
from lxml import etree

# Define example BoxShape, AddShape, MoveShape, Delete, and CollisionSettingInfo 

box = BoxShape(name="Table",frame="world",physical=True,x=2,y=1,z=1.2,translation = Translation(x=1.0, y =2.0, z=3.0),rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
addShape = AddShape(id = "123", shape = box)
moveShape = MoveShape(id = "123", translation = Translation(x=-1.0, y =0.0, z=4.0),rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
deleteShape = "123"
collision = CollisionSettingInfo(d_max = 0.1, r = 0.0, a_max = 2.0, time_budget = 100, timed = False)

# Read the xml file into a string
xml_file = '../../tests/panda.xml'
tree = etree.parse(xml_file)
xml_string = etree.tostring(tree).decode()

#print(xml_string)
# Instantiate a new solver
solver = Solver(
  urdf=xml_string, # Full urdf as a string
  objectives={
      # An example objectives
      "smoothness":SmoothnessMacroObjective(name="MySmoothnessObjective",weight=5,joints=True,origin=False,links=True),
      "collision": CollisionAvoidanceObjective(name="MyCollisionAvoidanceObjective", weight=5),
      "jointLimit": JointLimitsObjective(name="MyJointLimitObjective", weight=5)
      
  },
  root_bounds=[
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0), # Translational, (x, y, z)
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0)  # Rotational, (r, p, y)
  ],
  shapes=[
      box
  ],
  collision_settings = collision

)

# Run solve to get a solved state
state = solver.solve(goals= {},weights = {},time = 0.0,shape_updates = [addShape,moveShape,deleteShape])
# Log the initial statse
print(state.origin.as_dicts())
print(state.joints)
print(state.frames["panda_link0"])
print(state.proximity)

({'z': 0.0, 'y': 0.0, 'x': 0.0}, {'w': 1.0, 'y': 0.0, 'z': 0.0, 'x': 0.0})
{'panda_finger_joint1': 0.02, 'panda_joint6': 1.8675, 'panda_joint5': 0.0, 'panda_joint1': 0.0, 'panda_joint2': 0.0, 'panda_joint3': 0.0, 'panda_joint7': 0.0, 'panda_finger_joint2': 0.02, 'panda_joint4': -1.5708}
TransformInfo: world:{translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}, local: {translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}
[ProximityInfo: {shape1: "panda_link1", shape2: "panda_link2", distance: -0.04999999999594529, points: Some(([3.3428314619892455e-7, -8.11153227919803e-12, 0.39299999999573776], [-3.3428314619892455e-7, -7.957830575056057e-12, 0.3430000000042623])), physical: true, loss: 1.0499999999959453, average_distance: Some(1.0)}, ProximityInfo: {shape1: "panda_link3", shape2: "panda_link5", distance: -0.023348390002273036, points: Some(([0.09682043643243086, 1.9752255386862316e-13, 0.7013648061031561], [0.08967986660682825, 1.2992296